# Reglas de Slater-Condon

En este código vammos a implementar las reglas de Slater-Condon para construir nuestro Hamiltoniano multielectrónico, la idea es crear dos funciones, una para los operadores de un cuerpo y otro para los operadores de dos cuerpos.

La intención de este código también es poder hacerlo escalable, es decir, que no solo funcione para el caso del dímero del Hubbard (2 sitios y 2 electrones), sino que uno pueda elegir entre P sitios y N_e electrones.

In [ ]:
%matplotlib inline
import numpy as np
from qutip import *
import matplotlib.pyplot as plt
from sympy.utilities.iterables import multiset_permutations
from time import time

In [ ]:
def factorial(n):
    productoria = 1
    for i in range(2, n+1):
        productoria *= i
    return productoria

In [ ]:
def combinatorio(n, k):
    return factorial(n)//(factorial(n-k)*factorial(k)) 

La función $\textit{b(i)}$ crea una lista de $i=2*sitios$, cada elemnto de la lista asociado a un estado posible del sistema.

Ejemplo para $i=4$, el estado vacío sería:


$$\left| 0 \right\rangle=
\left(
\begin{matrix}
1 \\0 
\end{matrix} 
\right) \otimes
\left(
\begin{matrix}
1 \\0 
\end{matrix} 
\right) \otimes
\left(
\begin{matrix}
1 \\0 
\end{matrix} 
\right) \otimes
\left(
\begin{matrix}
1 \\0 
\end{matrix} 
\right)
$$

De tal forma que 
$$\left(
\begin{matrix}
1 \\0 
\end{matrix}
\right)$$ 

indica que no hay electrón en ese estado, y

$$\left(
\begin{matrix}
0 \\1 
\end{matrix}
\right)$$ 

indica que hay un electrón ese estado.

Para representar el estado, por ejemplo, $\left| 1\downarrow 1\uparrow \right\rangle$, se tendrá que

$$\left| 1\downarrow 1\uparrow \right\rangle=
\left(
\begin{matrix}
0 \\1 
\end{matrix} 
\right) \otimes
\left(
\begin{matrix}
0 \\1 
\end{matrix} 
\right) \otimes
\left(
\begin{matrix}
1 \\0 
\end{matrix} 
\right) \otimes
\left(
\begin{matrix}
1 \\0 
\end{matrix} 
\right)
$$

In [ ]:
def b(i): return [basis(2,0) for j in range(0, i)]

La función $\textit{base_comp(sites,N_electrones)}$ crea una lista con todos los diferentes estados posibles dependiendo de los sitios y los electrones.
A la lista inicial con ningún estado ocupado, dependiendo de los electrones del sistema, utilizando $\textit{create(2)}$ pasamos de ningún estado vacío, a tantos estados ocupados como electrones.

Luego la función $\textit{multiset_permutations()}$ devuelve una lista con todas las combinaciones de estados del sistema.

In [ ]:
def base_comp(Sites, N_electrones):
    q=b(2*Sites)
    for i in range(0,N_electrones):
        q[i]=create(2)*q[i]
    return list(multiset_permutations(q))
    

Esta función tiene de entrada dos estados del sistema y devuelve el número de electrones que son diferentes. Esta función reusltará útil en la aplicación de las reglas de Slater-Condon

In [ ]:
def e_diferentes(base1, base2, sites, n_electrones):
    c=0
    for i in range (0,2*sites):
        if (base1[i]==basis(2,1) and base2[i]==basis(2,1))==True:
            c+=1
    return n_electrones-c

Más tarde, en la aplicación de las reglas de Slater-Condon, en la condición de solo un electrón diferente, dependiendo si ambos estados diferentes están a la derecha o izquierda del estado coincidente hay que añadir un signo menos, debido a la naturaleza fermiónica de los electrones. 

El orden que hemos tomado para definir la base sería:$ [ 1\downarrow, 1\uparrow, 2\downarrow, 2\uparrow ]$


In [ ]:
def signo(base1, base2):
    perm=0
    i=0
    while(i<len(base1) and base1[i]==base2[i])==True:
        i+=1
    
    j=i+1
    while(j<len(base1) and base1[j]==base2[j])==True:
        if (base1[j]==basis(2,1) and base2[j]==basis(2,1))==True:
            perm+=1
        j+=1    
    if perm%2==0: return  1
    else: return (-1)
        

La función $\textit{spin_total}$ devuelve el espin del estado de entrada.

In [ ]:
def spin_total(base1):
    s=0
    for i in range (len(base1)):
        if(base1[i]==basis(2,1) and i%2==0):
            s+=-1
        elif (base1[i]==basis(2,1) and i%2!=0):
            s+=1         
    return s/2

El input son dos estados y el output es una lista con la posición de los estados que coinciden, por ejemplo, si tenemos de input: $\left| 1\downarrow 1\uparrow \right\rangle, \left| 1\downarrow 2\uparrow \right\rangle$, el output es una lista de un solo valor (en este caso), [0]. Si el input es $\left| 1\downarrow 1\uparrow \right\rangle, \left| 1\downarrow 1\uparrow \right\rangle$, la lista de será [0,1].


In [ ]:
def estados_coinciden(base1,base2):
    v=[]
    for i in range(len(base1)):
        if (base1[i]==basis(2,1) and base2[i]==basis(2,1))==True:
            v.append(i)
            
    return v

Y esta función es equivalente, pero con los estados diferentes.

In [ ]:
def estados_diferentes(base1,base2):
    v=[]
    for i in range(len(base1)):
        if(base1[i]!=base2[i]):
            v.append(i)
    return v

Este segmento del código es la implementación de las reglas de Slater-Condon para operadores de un cuerpo, "one-body operator". 

Primero vamos a ver que forma tiene un Hamiltoniano típico de uno y dos cuerpos:

$$\hat{H}=\sum_{mn\sigma} t_{mn}\hat{d}_{m\sigma}^\dagger\hat{d}_{n\sigma} +\sum_m v_m \hat{n}_{m}+\sum_{m\neq n} \gamma_{\left|m-n\right|} \hat{n}_m \hat{n}_n +\sum_m \gamma_0 \hat{n}_{m\uparrow} \hat{n}_{m\downarrow} $$

Y para el caso del modelo de Hubbard el Hamiltoniano que queda es:

$$\hat{H}=\sum_{mn\sigma} t_{mn}\hat{d}_{m\sigma}^\dagger\hat{d}_{n\sigma} +\sum_m v_m \hat{n}_{m}+\sum_m \gamma_0 \hat{n}_{m\uparrow} \hat{n}_{m\downarrow} $$

Donde los elementos de matriz de $t_{mn}$ y $v_{m}$ son los correspondientes a los estados de la base de la lattice $[ 1\downarrow, 1\uparrow, 2\downarrow, 2\uparrow ]$, que como veremos, nos facilitarán los cálculos.

Continuando con los operadores de un cuerpo, que tiene esta forma:

$$\hat{A}=\sum_{ij} a_{ij} \hat{d}^\dagger_i \hat{d}_j=\sum_{m\sigma, n\tau} a_{m\sigma, n\tau}\hat{d}^\dagger_{m\sigma}\hat{d}_{n\tau}$$

Tanto la parte cinética como de potencial externo son opradores de un cuerpo y cumplen las siguinetes reglas para operadores de un cuerpo:

1. Caso de estados multielectrónicos idénticos.

Teniendo el estado $\left| I \right\rangle=\left| \chi_{i(I,1)},...,\chi_{i(I,N)} \right\rangle$, el elemnto de matriz será:

$$\left\langle I \right| \hat{A} \left| I \right\rangle=\sum_{k=1}^N \left\langle \chi_{i(I,k)} \right| \hat{a} \left| \chi_{i(I,k)} \right\rangle$$

donde la base {$ \left| \chi_{i} \right\rangle $} no tiene porque ser la base de la lattice. De ser la base de la lattice, como en nuestro caso se tiene que:
$$\left\langle I \right| \hat{A} \left| I \right\rangle=\sum_{k=1}^N a_{i(I,k),i(I,k)}$$

Como la  notación puede llegar a ser un poco confusa, tomemos un ejemplo.

Tomemos $\left| I \right\rangle=\left| 1\downarrow 2\uparrow \right\rangle$, tenemos por tanto N=2 y, en este caso, se tendrá que $i(N,1)=1\downarrow$ y $i(N,2)=2\uparrow$ y como estamos trabajando en la base de la lattice, el elemento de matriz quedará:

$$\left\langle I \right| \hat{A} \left| I \right\rangle=a_{1\downarrow 1\downarrow}+a_{2\uparrow 2\uparrow}$$

El código utiliza $\textit{e_diferentes}$ para ver cuantos electrones en estados diferentes hay, si hay 0 entonces aplica la primera regla, para estados idénticos. Seguidamente, crea una lista ($\textit{e_c}$) con los ínidices de los estados de la base que coinciden através de la función $\textit{estados_coinciden}$. Estos índices en $\textit{e_c}$ son los ínidces $i(I,1), i(I,2)$. Luego se escoge el elemento de matriz con esos ínidices formamos la representación matricial.

2. Caso en el que se diferencian por un electrón
Sea el estado $\left| I \right\rangle=\left| \chi_{i(I,1)},...,\chi_{i(I,N)} \right\rangle$ y cambiamos el estado del elctrón $\chi_{i(I,k)}$ por otro estado $\chi_{a}$ tal que $\left| J \right\rangle=\hat{c}^\dagger_a \hat{c}_{i(I,k)}\left| I \right\rangle$. Luego, el elemento de matriz será:

$$\left\langle I \right| \hat{A} \left| J \right\rangle=\left\langle \chi_{i(I,k)} \right| \hat{a} \left| \chi_{a} \right\rangle$$

y si fuera la base de la lattice:

$$\left\langle I \right| \hat{A} \left| J \right\rangle=a_{i(I,k),a}$$

Es decir, el elemento de matriz  de los estados diferentes.

En este apartado hay que destacar que debido a la naturaleza fermiónica de los electrones hay que introducir, en algunos casos, signos menos debido a que hay que hacer permutaciones en el orden de los estados par lograr que, $\left| J \right\rangle=\hat{c}^\dagger_a \hat{c}_{i(I,k)}\left| I \right\rangle$.

Y volviendo al código, procedimiento es equivalente, si $\textit{e_diferentes}$ deveulve 1, entonces guardamos en la lista $\textit{e_f}$ los índices de los estados diferentes através de la función $\textit{estados_diferentes}$ y utilizando dichos índices compones el operador, de igual forma que en el caso anterior.

3. Caso de dos estados del electrón diferentes

Los elementos de matriz siempre son nulos.

In [ ]:
def one_body_operator(t,sites,n_electrones):
    
    base=base_comp(sites, n_electrones)
    
    e=0
    T=0
    
    
    for i in range (len(base)):
        for j in range(len(base)):
            if(e_diferentes(base[i],base[j],sites, n_electrones)==0):
                
                e_c=estados_coinciden(base[i],base[j])
                
                for l in range (0, n_electrones):
                    
                     
                    T+=t[e_c[l]][e_c[l]]*tensor(base[i])*tensor(base[j]).dag()
                
            elif (e_diferentes(base[i],base[j],sites, n_electrones)==1):
                
                e_f=estados_diferentes(base[i],base[j])
                
                T+=signo(base[i],base[j])*t[e_f[0]][e_f[1]]*tensor(base[i])*tensor(base[j]).dag()
    
    return T    

Vamos directamente a explicar en que consisten las reglas de Slater-Condon 
para operadores de dos cuerpos.

1. Caso de estados iguales

$$\left\langle I \right| \hat{W} \left| I \right\rangle=
\frac{1}{2}\sum_{k\neq l}[\left\langle \chi_{i(I,k)}\chi_{i(I,l)} \right| \hat{w} \left| \chi_{i(I,k)}\chi_{i(I,l)} \right\rangle-\left\langle \chi_{i(I,k)}\chi_{i(I,l)} \right| \hat{w} \left| \chi_{i(I,l)}\chi_{i(I,k)} \right\rangle]$$

Y si estamos utilizando la base de la lattice:

$$\left\langle I \right| \hat{A} \left| I \right\rangle=
\frac{1}{2}\sum_{k\neq l} w_{m(I,k),m(I,l)}$$

donde el índice $m$ indica el sitio en el que se encuentra el electrón. Tal y como está escrito el código $w$ está representada en la base de los estados de una partícula (lattice basis) y no en la base de los sitios en los que puede estar la partícula, pero más tarde se construye $w$ para que tome la forma de diagonal por cajas y equivalente a la notación usada.

En concreto la representación matricial de $w$ para la base de estados de una partícula, $ [ 1\downarrow, 1\uparrow, 2\downarrow, 2\uparrow ]$, será la siguiente:

$$\left(
\begin{matrix}
0&U&0&0\\
U &0 &0 &0\\
0&0&0&U\\
0&0&U&0
\end{matrix} 
\right)
$$

Ya que hemos tomado $\gamma_0=U$ y el resto de $\gamma_{\left|m-n\right|}=0$, para nuestro modelo en concreto.

Y en cuanto al código, recuperamos la función $\textit{estados_coinciden}$, para de la misma forma que en el caso de un cuerpo elegir los ínidces y componer el operador.

2. Caso de un estado diferente

Tomando nuevamente $\left| J \right\rangle=\hat{c}^\dagger_a \hat{c}_{i(I,k)}\left| I \right\rangle$

se tiene que:

$$\left\langle I \right| \hat{W} \left| J \right\rangle=
\sum_{l=1}^N[\left\langle \chi_{i(I,k)}\chi_{i(I,l)} \right| \hat{w} \left| \chi_{a}\chi_{i(I,l)} \right\rangle-\left\langle \chi_{i(I,k)}\chi_{i(I,l)} \right| \hat{w} \left| \chi_{i(I,l)}\chi_{a} \right\rangle]$$

En el caso de la ser la base de la lattice es nulo.

3. Caso de dos estados diferentes

Ahora sea $\left| J \right\rangle=
\hat{c}^\dagger_a \hat{c}_{i(I,k)}\hat{c}^\dagger_b \hat{c}_{i(I,l)}
\left| I \right\rangle$

Se tiene que

$$\left\langle I \right| \hat{W} \left| J \right\rangle=
\left\langle \chi_{i(I,k)}\chi_{i(I,l)} \right| \hat{w} \left| \chi_{a}\chi_{b} \right\rangle-\left\langle \chi_{i(I,k)}\chi_{i(I,l)} \right| \hat{w} \left| \chi_{b}\chi_{a} \right\rangle$$

De igual forma que en el caso anterior para la base de lattice es nulo.

4. Caso de más de dos electrones diferentes

El elemento de matriz es siempre nulo.

Como estamos utilzando la base de la lattice, este fragmento de código solo tiene implementado el primer caso. Una mejora sería implementar el resto de casos independientemente de la base utilizada.

In [ ]:
def two_body_operator(w,sites, n_electrones):
    
    base=base_comp(sites, n_electrones)
    
    W=0
    
    for i in range (len(base)):
        for j in range(len(base)):
            if(e_diferentes(base[i],base[j],sites, n_electrones)==0):
                
                e_c=estados_coinciden(base[i],base[j])
                for k in range(0, len(e_c)-1):
                    for l in range (k+1, len(e_c)):
                     
                        W+=0.5*w[e_c[k]][e_c[l]]*tensor(base[i])*tensor(base[j]).dag()
                        W+=0.5*w[e_c[l]][e_c[k]]*tensor(base[i])*tensor(base[j]).dag()
                        
    
    return W

En este segmento de código damos los valores de nuestro sistema: númreo
de electrones, número de sitios, el término cinético, t, el de potencial externo, v, y el potencial de interacción, w.

Además hay que restringir al subespacio de $\mathcal{H_2}$, ya que en todo momento estamos trabajando en el espacio de Fock, $\mathcal{F}=\mathcal{H_0}\oplus\mathcal{H_1}\oplus\mathcal{H_2}\oplus\mathcal{H_3}\oplus\mathcal{H_4}$ y dentro del subespacio $H_2$, al que está restringido por los estados con espines antiparalelos.

Para ello creamos las matrices A y B.

In [ ]:
N_e=2
P=2
t = [[0 for x in range(2*P)] for y in range(2*P)]
v= [[0 for x in range(2*P)] for y in range(2*P)]
w=[[0 for x in range(2*P)] for y in range(2*P)]

t[0][2]=t[2][0]=t[1][3]=t[3][1]=-0.5

v_ext=np.linspace(0, 10, 50)

U=[0.2, 1, 2, 5, 10]

base=[basis(4,i) for i in range (0,4)]
n_1=2*base[0]*base[0].dag()+base[1]*base[1].dag()+base[2]*base[2].dag()
n_2=2*base[3]*base[3].dag()+base[1]*base[1].dag()+base[2]*base[2].dag()
n=[n_1, n_2]

A=0

base_total=base_comp(P,N_e)

for i in range (combinatorio(2*P,N_e)):
    A+=tensor(base_total[i])*basis(combinatorio(2*P,N_e),i).dag()

In [ ]:
E=[]
delta_N=[]
t_0=time()

h_1=one_body_operator(t,P,N_e)
for l in range (0,len(U)):
    
    e=[]
    delta_n=[]
    
    w[0][1]=w[1][0]=w[2][3]=w[3][2]=U[l]
    
    
    h_2=two_body_operator(w,P,N_e)
    
    for k in range (0, len(v_ext)):
    
        v[0][0]=v[1][1]=v_ext[k]
        v[2][2]=v[3][3]=-v_ext[k]
        
        h_3=one_body_operator(v,P,N_e)
        
        H=h_1+h_2+h_3
        H_2=A.dag()*H*A
        evals, ekets= H_2.eigenstates()
        e.append(evals[0])
        #delta_n.append(abs((n[0]-n[1]).matrix_element(ekets,ekets)))
    
    E.append(e)
    #delta_N.append(delta_n)
    
t_1=time()-t_0

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

plt.ylim(-10, 0)
plt.xlim(0, 20)

ax.plot(2*v_ext, E[0])
ax.plot(2*v_ext, E[1])
ax.plot(2*v_ext, E[2])
ax.plot(2*v_ext, E[3])
ax.plot(2*v_ext, E[4])
ax.legend(("U=0.2", "U=1", "U=2", "U=5", "U=10"))
ax.set_xlabel('Δv')
ax.set_ylabel('E')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

plt.ylim(0, 2)
plt.xlim(0, 20)

ax.plot(2*v_ext, delta_N[0])
ax.plot(2*v_ext, delta_N[1])
ax.plot(2*v_ext, delta_N[2])
ax.plot(2*v_ext, delta_N[3])
ax.plot(2*v_ext, delta_N[4])
ax.legend(("U=0.2", "U=1", "U=2", "U=5", "U=10"))
ax.set_xlabel('Δv')
ax.set_ylabel('|Δn|')

plt.show()